In [ ]:
import os
import sys
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# set warnings
import warnings
warnings.simplefilter(action='ignore', category = Warning)

# add parent folder path to the namespace
sys.path.append(os.path.dirname(os.getcwd()))

# import modules and components
from utils.preprocessing import dataset_from_images
from utils.generators import DataGenerator, TensorDataSet
from utils.validation import ModelValidation
from utils.models import Inference
import utils.global_paths as globpt
import configurations as cnf

# specify relative paths from global paths and create subfolders
images_path = os.path.join(globpt.data_path, 'images')
cp_path = os.path.join(globpt.train_path, 'checkpoints')
os.mkdir(images_path) if not os.path.exists(images_path) else None
os.mkdir(cp_path) if not os.path.exists(cp_path) else None

# 1. Load data and model

In [ ]:
inference = Inference(cnf.seed) 

# load the model for inference and print summary
model, parameters = inference.load_pretrained_model(cp_path)
model_path = inference.folder_path
model.summary(expand_nested=True)

# load and reprocess data
filepath = os.path.join(model_path, 'preprocessing', 'train_data.csv')                
train_data = pd.read_csv(filepath, sep=';', encoding='utf-8')
filepath = os.path.join(model_path, 'preprocessing', 'test_data.csv')                
test_data = pd.read_csv(filepath, sep=';', encoding='utf-8')

### 1.1 Create generator and datasets

In [ ]:
# regenerate paths
train_data = dataset_from_images(images_path, dataset=train_data)
test_data = dataset_from_images(images_path, dataset=test_data)

# initialize the images generator for the train and test data, and create the 
# tf.dataset according to batch shapes
train_generator = DataGenerator(train_data, 20, cnf.picture_shape, 
                                augmentation=False, shuffle=True)
test_generator = DataGenerator(test_data, 20, cnf.picture_shape, 
                               augmentation=False, shuffle=True)

# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators 
datamaker = TensorDataSet()
train_dataset = datamaker.create_tf_dataset(train_generator)
test_dataset = datamaker.create_tf_dataset(test_generator)

# 2. Model performance evaluation

### 2.1 Evaluation of loss and metrics

In [ ]:
validator = ModelValidation(model)

# create subfolder for evaluation data
eval_path = os.path.join(model_path, 'evaluation') 
os.mkdir(eval_path) if not os.path.exists(eval_path) else None

# evluate the model on both the train and test dataset
train_eval = model.evaluate(train_dataset, batch_size=25, verbose=1)
test_eval = model.evaluate(test_dataset, batch_size=25, verbose=1)

print('\nTrain dataset:')
print(f'Loss: {train_eval[0]}')    
print(f'Metric: {train_eval[1]}')  
print('\nTest dataset:')
print(f'Loss: {test_eval[0]}')    
print(f'Metric: {test_eval[1]}') 

### 2.2 Reconstruction evaluation

Compare reconstructed images to original pictures to qualitatively evaluate the performance of the FeXT autoencoder model

In [ ]:
# perform visual validation for the train dataset (initialize a validation tf.dataset
# with batch size of 10 images)
print('Visual reconstruction validation: train dataset\n')
plot_name = 'visual_validation_train'
validation_batch = train_dataset.unbatch().batch(10).take(1)
for images, labels in validation_batch:
    recostructed_images = model.predict(images, verbose=0)
    validator.visualize_reconstructed_images(images, recostructed_images, plot_name, eval_path)

# perform visual validation for the test dataset (initialize a validation tf.dataset
# with batch size of 10 images)
print('Visual reconstruction validation: test dataset\n')
plot_name = 'visual_validation_test'
validation_batch = test_dataset.unbatch().batch(10).take(1)
for images, labels in validation_batch:
    recostructed_images = model.predict(images, verbose=0) 
    validator.visualize_reconstructed_images(images, recostructed_images, plot_name, eval_path)

Visualize the original image and the reconstructed images, together with the corresponding features vector. The raw vector has shape 8x8x512 and is reshaped to be 64x512

In [ ]:
single_image_batch = train_dataset.unbatch().batch(1).take(1)

# isolate the encoder from the autoencoder model
encoder_input = model.get_layer('input_1')  
encoder_output = model.get_layer('fe_xt_encoder')  
encoder_model = keras.Model(inputs=encoder_input.input, outputs=encoder_output.output)

# extract features vector
recostructed_image = model.predict(single_image_batch, verbose=0) 
extracted_features = encoder_model.predict(single_image_batch, verbose=0)
reshaped_features = extracted_features.reshape(64, 512)

plot_name = 'visual_features_vector'
for original_image, label in single_image_batch:
    validator.visualize_features_vector(original_image, reshaped_features, recostructed_image,
                                        plot_name , eval_path)